In [1]:

import numpy as np
import pandas as pd
import re
import sqlite3
from sqlite3 import Error

import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize
# For Natural Language Processing
import regex as re
import unidecode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

# For classification modeling
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# For evaluation
from sklearn.metrics import confusion_matrix

# For wordclouds
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

pd.set_option('display.max_colwidth', -1)

In [2]:
#read the csv
wine_data = pd.read_csv("winemag-data-130k-v2.csv")

In [3]:
# Create your connection.
conn = sqlite3.connect('db\wine_data.sqlite')
c = conn.cursor()

In [4]:
#creat a table in the database
wine_data.to_sql('wine_data', conn, if_exists = "replace")
#read the table in the database
wine_df = pd.read_sql('Select * from wine_data', conn)

/Users/isaacnwang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [5]:
#Drop the duplicate descriptions
wine_df = wine_df.drop_duplicates('description')

#drop null prices
wine_df = wine_df.dropna(subset=['price'])

#filter the dataframe to include only varieties with more than 200 reviews
wine_df = wine_df.groupby('variety').filter(lambda x: len(x) > 200)

#create a column named color
wine_df["color"] = ""

#update the database
wine_df.to_sql('wine_data', conn, if_exists = "replace")

In [6]:
pd.read_sql('SELECT variety, count(variety) as vcount FROM wine_data group by variety', conn)

,variety,vcount
0,Aglianico,274
1,Albariño,432
2,Barbera,545
3,Bordeaux-style Red Blend,4958
4,Bordeaux-style White Blend,635
5,Cabernet Franc,1196
6,Cabernet Sauvignon,8760
7,Carmenère,525
8,Champagne Blend,1134
9,Chardonnay,10237


In [66]:
#used to update the database with the wine color. Maunally updated each wine variety.
c.execute("update wine_data set color = 'red' where variety = 'Zinfandel' ")

#commit the update to the database so it saves
conn.commit()

In [67]:
#verify data saved as expected
pd.read_sql("select color, count(color) from wine_data group by color" ,conn)

,color,count(color)
0,,289
1,red,62571
2,white,37401


In [68]:
#remove all the records without a color
wine_df = pd.read_sql("select country, description,price,province,title,variety,winery, color  from wine_data where color in ('red', 'white')", conn)
wine_df.to_sql('wine_data', conn, if_exists = "replace")
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99972 entries, 0 to 99971
Data columns (total 8 columns):
country        99939 non-null object
description    99972 non-null object
price          99972 non-null float64
province       99939 non-null object
title          99972 non-null object
variety        99972 non-null object
winery         99972 non-null object
color          99972 non-null object
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


In [69]:
#drop the rest of the NA values
wine_df = wine_df.dropna()
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99939 entries, 0 to 99971
Data columns (total 8 columns):
country        99939 non-null object
description    99939 non-null object
price          99939 non-null float64
province       99939 non-null object
title          99939 non-null object
variety        99939 non-null object
winery         99939 non-null object
color          99939 non-null object
dtypes: float64(1), object(7)
memory usage: 6.9+ MB


In [70]:
#update the database
wine_df.to_sql('wine_data', conn, if_exists = "replace")

In [71]:
#verify the data
wine_df.head()

,country,description,price,province,title,variety,winery,color
0,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red
1,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,white
2,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,white
3,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,red
4,France,"This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.",24.0,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,white


In [72]:
wine_df.shape

(99939, 8)

In [73]:
wine_df = wine_df.drop_duplicates('description') 
wine_df.shape

(99939, 8)

In [74]:
wine_df.isnull().sum()

country        0
description    0
price          0
province       0
title          0
variety        0
winery         0
color          0
dtype: int64

Data looks cleaned. Time to perform text preprocessing.

In [78]:
wine_df.to_csv('Wine.csv')

# Text PreProcessing

In [82]:
# Preprocessing function
def reviews_to_words(raw_review):
    
    # Get rid of accents
    unaccented = unidecode.unidecode(raw_review)
    
    # Get rid of punctuation
    letters_only = re.sub("[^a-zA-Z]", " ", unaccented)
    
    # Get all lowercase words
    words = letters_only.lower().split()
    
    # Instantiate and run Lemmatizer 
    lemmatizer = WordNetLemmatizer()
    tokens_lem = [lemmatizer.lemmatize(i) for i in words]
    
    # Remove stop words
    stops = set(stopwords.words('english'))
    stops.add('wine')
    meaningful_words = [w for w in words if not w in stops]
    
    # Join into string and return the result.
    return(" ".join(meaningful_words))

In [83]:
# Clean all titles
total_reviews = wine_df.shape[0]
clean_reviews = []

for review in wine_df['description']:
    # Convert to words, then append to clean_train
    clean_reviews.append(reviews_to_words(review))

In [84]:
wine_df = wine_df.assign(clean_review = clean_reviews)
wine_df.head(3)

,country,description,price,province,title,variety,winery,color,clean_review
0,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red,ripe fruity smooth still structured firm tannins filled juicy red berry fruits freshened acidity already drinkable although certainly better
1,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,white,tart snappy flavors lime flesh rind dominate green pineapple pokes crisp acidity underscoring flavors stainless steel fermented
2,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,white,pineapple rind lemon pith orange blossom start aromas palate bit opulent notes honey drizzled guava mango giving way slightly astringent semidry finish


In [81]:
#calculate the frequency
word_frequency = pd.Series(' '.join(wine_df['clean_review']).split()).value_counts()[:30]
word_frequency

wine          60055
flavors       50389
fruit         37984
aromas        29207
palate        28889
finish        27266
acidity       25566
tannins       23477
drink         23302
cherry        23296
black         22544
ripe          20349
red           16553
oak           14810
notes         14670
spice         14408
dry           13953
rich          13394
nose          13142
berry         12748
fresh         12453
full          12401
plum          11547
sweet         10601
soft          10514
blend         10463
well          10268
apple         10143
blackberry    9983 
fruits        9818 
dtype: int64

Going to add wine to my list of stopwords in my function

In [85]:
#calculate the frequency
word_frequency = pd.Series(' '.join(wine_df['clean_review']).split()).value_counts()[:30]
word_frequency

flavors       50389
fruit         37984
aromas        29207
palate        28889
finish        27266
acidity       25566
tannins       23477
drink         23302
cherry        23296
black         22544
ripe          20349
red           16553
oak           14810
notes         14670
spice         14408
dry           13953
rich          13394
nose          13142
berry         12748
fresh         12453
full          12401
plum          11547
sweet         10601
soft          10514
blend         10463
well          10268
apple         10143
blackberry    9983 
fruits        9818 
light         9778 
dtype: int64

Much better. Time to test train split and vectorize

In [86]:
wine_df.to_csv('Cleaned_Wine.csv')